# Building a database for crime reports

## Introduction

Our team has been tapped by the Boston governement to build a database for storing data related to crimes that occurred in Boston. This dataset is available in a file called boston.csv. 

The requirements given to us is to create a database named crimes_db with appropriate datatypes for storing the data from the boston.csv file. It will be placed inside a schema named crimes. A table will also be created. There will also be readonly and readwrite groups with the appropriate privileges created. Finally, users will be created for each of these groups.

## Creating the Crime Database

We will start by creating a database for storing the crime data as well as a schema for containing the tables. A database can be created using the following command: CREATE DATABASE database_name;

Inside this database, we will create a schema to keep the data organized. 


In [1]:
import psycopg2
conn = psycopg2.connect("dbname=dq user=dq")

# To create database we set autocommit to true
conn.autocommit = True
cur = conn.cursor()

# Create the crime_db database
cur.execute("CREATE DATABASE crime_db;")
conn.close()

ProgrammingError: database "crime_db" already exists


In [2]:
# Now let us connect to crime_db with user dq
conn = psycopg2.connect("dbname=crime_db user=dq")
cur = conn.cursor()


# To create a schema named crimes;
cur.execute("CREATE SCHEMA crimes;")

## Obtaining the Column Names and Sample

Since we now have a database and a schema, we're ready to start creating tables. First, let's gather some data about our crime dataset so that we can easily select the right datatypes to use in our table.

We will start by reading the column names from the boston.csv file as well as the first row. This way we will have them at hand throughout this guided project for easy reference

In [3]:
# We can read a csv file using the csv module as follows
import csv
with open('boston.csv') as file:
    reader = csv.reader(file)
    col_headers = next(reader)
    first_row = next(reader)

## Creating an Auxiliary Function

Before creating a table to store the crime data, we need to identify the proper datatypes for the columns. To help with that, we'll create a function '**get_col_set()**', that, given the name of a CSV file and a column index (starting at 0), computes a Python set with all distinct values contained in that column.

This function will be useful for two reasons:

1. Checking whether an enumerated datatype might be a good choice for representing a column

2. Computing the maximum length of any text-like column to select appropriate sizes for VARCHAR columns

In [4]:
# To create a get_col_set(csv_filename, col_index)

def get_col_set(csv_filename, col_index):
    import csv
    values = set()
    with open(csv_filename, 'r') as f:
        next(f)
        reader = csv.reader(f)
        for row in reader:
            values.add(row[col_index])
        return values

# To compute the number of different values each column contains     
for i in range(len(col_headers)):
    values = get_col_set("boston.csv", i)
    print(col_headers[i], len(values), sep='\t')    

incident_number	298329
offense_code	219
description	239
date	1177
day_of_the_week	7
lat	18177
long	18177


## Finding the Maximum Length

It is important to know the longest word in any column containing textual data. We can actually use the previous function for computing this:

1. Use the **get_col_set()** function to get the distinct values of that column into a variable named **col_values**.

2. Iterate over the elements of **col_values**, and find the longest one.

There are two textual columns in the dataset, namely, the description and **day_of_the_week** columns. However, the day of the week contains only seven values, one for each day. We can tell that the longest of them is Wednesday without any computation. We'll compute the maximum length of each value in the description column.

In [5]:
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


In [6]:
descriptions = get_col_set('boston.csv', 2)
max_len = 0
for description in descriptions:
    max_len = max(max_len, len(description))
print(max_len)

58


## Creating the Table

Let's create a table to store the Boston crime data. We'll make reference to the information that we have gathered namely, the number of different values of each column and the maximum length of the string columns.

In [7]:
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


The same names will be used for the column headers.

The following are the values for each column:
* incident_number 298329
* offense_code       219
* description        239
* date	          1177
* day_of_the_week      7
* lat              18177
* long	         18177

By printing the first_row, we see the kind of data we have;
* integer numbers
* integer numbers
* string
* date
* string
* decimal number
* decimal number

The column with a smale range of values is the **day_of_the_week** column, hence we will only create an enumerated datatype for this column. Another column worth considering is **offense_code** since there is probably a limited set of possible offense codes.

The **offense_code** column has size at most 59. To be on the safe side, we will limit the size of the description to 100 and use the VARCHAR(100) datatype.

The lat and long columns need to hold quite a lot of precision, hence we will use the decimal type.

In [8]:
# Create the enumerated dataype for representing the weekday

cur.execute("""
            CREATE TYPE weekday AS ENUM ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday');
            """)

# Create the table
cur.execute("""
            CREATE TABLE crimes.boston_crimes (
            incident_number INTEGER PRIMARY KEY,
            offense_code INTEGER,
            description VARCHAR(100),
            date DATE,
            day_of_the_week weekday,
            lat decimal,
            long decimal);""")


## Loading the Data

Having created the table, we can load the data into it. We will use the **cursor.copy_expert()** method to do this.

In [9]:
with open("boston.csv") as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)
cur.execute("SELECT * FROM crimes.boston_crimes")

# To ensure the number of rows were loaded, we shall print them.
print(len(cur.fetchall()))

298329


## Revoking Public Privileges

Now we will handle users. The goal is to create the two user groups, the readonly and readwrite. By following the least privilege principle, the first step in doing so is to make sure that there are no privileges inherited from the public group and on the public schema.

We can do this by issuing the following two commands:

REVOKE ALL ON SCHEMA public FROM public;

REVOKE ALL ON DATABASE database_name FROM public;

In [10]:
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")

cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

## Creating User Groups

We've made sure that we aren't going to inadvertently inherit privileges from the public group. The next step is to create our two user groups, which are readonly and readwrite groups.

In [11]:
cur.execute("CREATE GROUP readonly NOLOGIN;")
cur.execute("CREATE GROUP readwrite NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")
cur.execute("GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;")


## Creating Users

We are nearly done with setting up our crime database. The only thing that we need to do is create users. Let's complete the project by creating one user in each group. We will need to create each user and then assign them to each group. We can create a user using the following command:

CREATE USER user_name WITH PASSWORD 'some_password';

In [12]:
# To create a user named data_analyst with password secret1
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("GRANT data_analyst TO readonly;")

# To create a user named data_scientist with password secret2
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute("GRANT data_scientist TO readwrite;")

conn.commit()

## Testing

Now that we have finished setting up the database, let us test that everything is configured as expected. We can query Postgres internal tables to see this information.

We can use SQL queries to check whether the objects have been created and that users and groups have the right privileges. This requires us to know the Postgres internal tables. We can query the pg_user, pg_roles table to inspect privileges related to the database and the information_schema.table_privileges table to inspect table privileges.

In information_schema.table_privileges table we will check privileges realted to type of SQL queries on the table. We will list the group name and the SQL queries, users in each group are allowed to make.

In [13]:
# Helper function to fetch and print in readable format
def fetch_all(cursor):
    values = cursor.fetchall()
    for value in values:
        print(value)
    print('\n')
    return values

In [14]:
# Let's Check the users and privileges from "pg_user" and "information_schema.table_privileges"
cur.execute("""
    SELECT * 
    FROM pg_user
    WHERE usename IN ('data_analyst', 'data_scientist');
""")
users = fetch_all(cur)

# Privileges realted to readonly and readwrite groups
cur.execute('''SELECT grantor, grantee, privilege_type 
                FROM information_schema.table_privileges 
                WHERE grantee IN ('readonly', 'readwrite');''')
privil = fetch_all(cur)

('data_analyst', 16497, False, False, False, False, '********', None, None)
('data_scientist', 16498, False, False, False, False, '********', None, None)


('dq', 'readonly', 'SELECT')
('dq', 'readwrite', 'INSERT')
('dq', 'readwrite', 'SELECT')
('dq', 'readwrite', 'UPDATE')
('dq', 'readwrite', 'DELETE')




In [15]:
# I will also check "pg_roles" (It shows both users and groups)
cur.execute("""
    SELECT * 
    FROM pg_roles
    WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');
""")

pg_roles = fetch_all(cur)

# Privileges realted to boston_crimes table
cur.execute('''SELECT grantor, grantee, privilege_type 
                FROM information_schema.table_privileges 
                WHERE table_name = 'boston_crimes';''')
privil_table = fetch_all(cur)

('readonly', False, True, False, False, False, False, False, -1, '********', None, None, 16495)
('readwrite', False, True, False, False, False, False, False, -1, '********', None, None, 16496)
('data_analyst', False, True, False, False, False, True, False, -1, '********', None, None, 16497)
('data_scientist', False, True, False, False, False, True, False, -1, '********', None, None, 16498)


('dq', 'dq', 'INSERT')
('dq', 'dq', 'SELECT')
('dq', 'dq', 'UPDATE')
('dq', 'dq', 'DELETE')
('dq', 'dq', 'TRUNCATE')
('dq', 'dq', 'REFERENCES')
('dq', 'dq', 'TRIGGER')
('dq', 'readonly', 'SELECT')
('dq', 'readwrite', 'INSERT')
('dq', 'readwrite', 'SELECT')
('dq', 'readwrite', 'UPDATE')
('dq', 'readwrite', 'DELETE')


